<h1>Testing the idea for frag clustering</h1>
<p>The idea is to use a two stage process to try and cluster all of the remaining peaks based on unknown fragments</p>

In [323]:
%load_ext autoreload
%autoreload 2
import adduct_cluster as ac

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [324]:
a_c = ac.AdductCluster(mass_tol = 5, mh_biggest = False)
# a_c.init_from_file('/Users/simon/git/metabolomics_tools/discretisation/java_src/James/txt/std1_Beth20512.peakml.txt')
a_c.init_from_file('test_small.txt')

Loaded 22 transformations from mulsubs/mulsub2.txt
Loaded 299 peaks from test_small.txt
Created 299 clusters
16 peaks to be re-sampled in stage 1


In [267]:
a_c.reset_counts()
a_c.multi_sample(1000)
a_c.compute_posterior_probs()


<p>Run VB</p>

In [329]:
a_c.init_vb()
for i in range(100):
    a_c.vb_step()


for peak in a_c.todo:
    print "Peak: {}".format(peak.mass)
    for poss in a_c.possible[peak]:
        print "\t {} {}".format(poss.transformation.name,poss.prob)

Peak: 192.107486901
	 M+H 0.00117959532246
	 M+HC13 0.998820404678
Peak: 236.073860848
	 M+H 0.959213860212
	 M+NH4 0.0407861397881
Peak: 128.070316369
	 M+H 0.00141108546645
	 M+NH4 0.937009308728
	 M+ACN+H 0.0615796058051
Peak: 240.164997455
	 M+H 0.00115519748724
	 M+HC13 0.998844802513
Peak: 146.029662593
	 M+H 0.00213374445456
	 M+ACN+H 0.997866255545
Peak: 146.029681836
	 M+H 0.000766964031869
	 M+ACN+H 0.999233035968
Peak: 77.9988036717
	 M+H 0.00257438082678
	 M+ACN+2H 0.997425619173
Peak: 87.0040552883
	 M+H 0.961575513343
	 M+H+NH4 0.0272290776649
	 M+ACN+2H 0.0111954089918
Peak: 87.0040613254
	 M+H 0.836842986337
	 M+H+NH4 0.0261716572048
	 M+H+NH4 0.0131826415326
	 M+ACN+2H 0.113244154784
	 M+ACN+2H 0.0105585601416
Peak: 199.000620516
	 M+H 0.001070567989
	 M+ACN+H 0.998929432011
Peak: 102.970095269
	 M+H 0.998729409239
	 M+H+Na 0.00127059076051
Peak: 143.996775021
	 M+H 0.000881948799419
	 M+ACN+H 0.999118051201
Peak: 197.000588811
	 M+H 0.000832814042441
	 M+ACN+H 0.99916

<p>assign all peaks to their most likely cluster</p>

In [222]:
a_c.map_assign()

In [223]:
singletons = []
for peak in a_c.peaks:
    if a_c.Z[peak].cluster.N == 1:
        singletons.append(peak)
print len(singletons)

2495


<p>Loop over all singletons and find their mass difference with M+H in the clusters within `a_c.rt_tol`</p>

In [235]:
import numpy as np
mass_differences = []
for peak in singletons:
    for cluster in a_c.clusters:
        if cluster.N > 0:
            if peak is cluster.mHPeak:
                continue
            elif np.abs(peak.rt - cluster.mHPeak.rt) < a_c.rt_tol:
                mass_differences.append(cluster.mHPeak.mass - peak.mass)

In [236]:
mass_differences = np.array(mass_differences)
mass_differences.sort()

In [250]:
histogram = {}
start_pos = 0
finished = False
threshold = 7
biggest = []
while not finished:
    new_key = str(mass_differences[start_pos])
    histogram[new_key] = 1
    search_pos = start_pos + 1
    search_finished = False
    while not search_finished:
        if search_pos >= len(mass_differences):
            search_finished = True
            finished = True
            break
        if np.abs((mass_differences[start_pos] - mass_differences[search_pos]) / mass_differences[start_pos])*1e6 < 2*a_c.mass_tol:
            histogram[new_key] += 1
            if histogram[new_key] > biggest:
                biggest = histogram[new_key]
            search_pos += 1
        else:
            if histogram[new_key] >= threshold:
                biggest.append(new_key)
            search_finished = True
            start_pos = search_pos


In [251]:
import sys
sys.path.append('/Users/simon/git/MS2LDA')
from efcompute import ef_assigner as ef

In [330]:
e = ef.ef_assigner(verbose = False,do_rule_8 = True)

TypeError: __init__() got an unexpected keyword argument 'max_ppm'

In [332]:
for m in biggest:
    a = e.find_formulas([np.abs(float(m))],ppm = 20)
    print "{}: {} -> {}".format(m,histogram[m],a[1][0])

-102.031721349: 9 -> C4H6O3
-84.9903334528: 7 -> CHNSC132
-72.9860838443: 7 -> None
-72.9676095483: 9 -> None
-64.9787826901: 7 -> None
-64.4776156492: 7 -> None
-61.9705080943: 8 -> None
-44.4656454236: 10 -> None
-42.010609232: 9 -> C2H2O
-28.5019468753: 8 -> None
-27.9950677767: 10 -> CO
-23.0160769694: 10 -> None
-22.9894836263: 7 -> None
-20.5135116693: 10 -> None
-20.5133031056: 27 -> None
-20.0117451317: 10 -> None
-18.0105716206: 16 -> H2O
-17.5048454554: 9 -> None
-16.0312246502: 7 -> CH4
-15.9776337864: 7 -> None
-10.9971604691: 7 -> None
-8.4903115232: 9 -> None
-7.98623981951: 7 -> None
7.98613732831: 7 -> None
8.49020522892: 9 -> None
10.9970681576: 8 -> None
14.0156079575: 7 -> CH2
15.9775787296: 7 -> None
17.5047790194: 9 -> None
18.0103572162: 12 -> H2O
20.0116115364: 9 -> None
20.513012233: 21 -> None
20.5132617977: 15 -> None
23.0157704394: 9 -> None
27.9946172953: 11 -> CO
28.0310528053: 7 -> C2H4
28.5019376859: 8 -> None
42.0103383843: 8 -> C2H2O
44.4652498141: 10 -

In [224]:
PROTON_MASS = 1.00727645199076
ATOM_NAME_LIST = ['C', 'H', 'N', 'O', 
                  'P', 'S', 'C13', 'F', 
                  'Cl']

ATOM_MASS_LIST = [12.00000000000,   1.00782503214,    14.00307400524,   15.99491462210, 
                  30.97376151200,   31.97207069000,   13.00335483780,   18.99840325000, 
                  34.96885271000]
ATOM_VALENCE_LIST = [4, 1, 3, 2, 3, 2, 4, 1, 1]

ATOM_MASSES = dict(zip(ATOM_NAME_LIST, ATOM_MASS_LIST))

In [225]:
ACN = {'C':2,'H':3,'N':1}
acn_mass = 0.0
for atom in ACN:
    acn_mass += ACN[atom]*ATOM_MASSES[atom]
print acn_mass, acn_mass + PROTON_MASS

41.0265491017 42.0338255537
